[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CoolandHot/DATA5709Capstone/blob/main/DATA5709_Capstone_Shinyapp_online.ipynb)

## Create Shiny app files

Write to *.r file

In [1]:
%%writefile shiny_app_colab.r

####################################
# User Interface                   #
####################################
library(shiny)
library(shinythemes)
library(shinycssloaders)
library(dplyr)
library(sf)
library(lubridate)
library(data.table)
library(ggplot2)
library(plotly)

# NYshape <- readRDS(file = "/content/drive/Shareddrives/Education/5709_Capstone/shinyRDS/NYshape_colab.RDS")
# historical_demand <- readRDS(file = "/content/drive/Shareddrives/Education/5709_Capstone/shinyRDS/historical_demand.rds")
# historical_supply <- readRDS(file = "/content/drive/Shareddrives/Education/5709_Capstone/shinyRDS/historical_supply.rds")
# pred_demand <- readRDS(file = "/content/drive/Shareddrives/Education/5709_Capstone/shinyRDS/pred_demand.rds")
# pred_supply <- readRDS(file = "/content/drive/Shareddrives/Education/5709_Capstone/shinyRDS/pred_supply.rds")
# distance_matrix <- readRDS(file = "/content/drive/Shareddrives/Education/5709_Capstone/shinyRDS/distance_matrix.rds")

load("shape_hist_pred_dist.RData")


tmp <- NYshape %>% as.data.frame() %>% select(zone, LocationID)
location.name.to.id <- tmp$LocationID
names(location.name.to.id) <- tmp$zone

ui <- fluidPage(theme = shinytheme("united"),
                navbarPage("New York City taxi order historical review and prediction:",
                           tabPanel("Map",
                                    fluidRow(
                                      HTML("<h3>Input parameters</h4>"),
                                      column(12,
                                             sliderInput("Timestamp", 
                                                         label = "Time of a day", 
                                                         value = ymd_hm("2019-12-01 09:00"), 
                                                         min = ymd_hm("2019-12-01 00:00"), 
                                                         max = ymd_hm("2019-12-1 23:59"),
                                                         timeFormat = "%H:%M", timezone = "+0000",
                                                         step = 60*10, width = '100%')),
                                      column(2,
                                             radioButtons("wDay", label = "Weekdays", 
                                                          choices = c("Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"),
                                                          selected = "Sun")),
                                      column(2,
                                             checkboxGroupInput("Vendor", label = "Which vendor/vendors?", 
                                                                choices = list("Uber"="uber.label", "Lyft"="lyft.label", "Via"="via.label", "Green"="green.label", "Yellow"="yellow.label"),
                                                                selected = c("uber.label", "lyft.label", "via.label", "green.label", "yellow.label"))),
                                      
                                      # column(4, 
                                      #        actionButton("browser", "Debug")
                                      #        ),# for debugging
                                      
                                      column(3,
                                             tabsetPanel(type = "tabs",
                                                         tabPanel("History", 
                                                                  HTML("<p>Directly press the submit button below.</p>"),
                                                                  actionButton("Hist_submitbutton", 
                                                                               "Submit", 
                                                                               class = "btn btn-primary")
                                                         ),
                                                         tabPanel("Prediction", 
                                                                  selectInput("pickupID",
                                                                              label = "Where are you?",
                                                                              choices = location.name.to.id,
                                                                              selected = 76, width = '100%'),
                                                                  actionButton("Pred_submitbutton", 
                                                                               "Submit", 
                                                                               class = "btn btn-primary")
                                                         )
                                             )# tabsetPanel
                                      )
                                      
                                    ), # common fluidRow
                                    # column
                                    
                                    HTML("<h3>Detail outputs</h3>"), # message title
                                    HTML("<p><b>Total quantity</b> is the sum of both demands and supplies which includes uber, lyft, via yellow and green taxi.</p>"),
                                    plotlyOutput("distMapPlot")%>% withSpinner(color="#0dc5c1"), # show loading animation
                                    htmlOutput('suggestions_curr'), # current time frame suggestion message
                                    plotOutput("distStatPlot_curr")%>% withSpinner(color="#0dc5c1"),
                                    HTML("<br>"),
                                    htmlOutput('suggestions_future'), # future suggestion message
                                    plotOutput("distStatPlot_future")%>% withSpinner(color="#0dc5c1")
                                    
                           ), # tabPanel(), Home
                           
                           tabPanel("About", 
                                    titlePanel("About"), 
                                    # div(htmlOutput("aboutHTML"), align="justify")
                                    div(uiOutput("aboutHTML"))
                           ) #tabPanel(), About
                           
                ) # navbarPage()
) # fluidPage()







##############################
# Predefined functions
##############################
prepare_map_data <- function(picked.wDay, picked.hm, Vendor){
  # 1. filter by selected values
  # 2. combine
  
  filtered_supply <- pred_supply[weekNum == picked.wDay & DO_time == picked.hm, ] %>%
    tidyr::unite("label.txt", all_of(Vendor), sep = "\n") %>%
    .[, .(DOLocationID, green, lyft, uber, via, yellow, total, label.txt)]
  filtered_demand <- pred_demand[weekNum == picked.wDay & PU_time == picked.hm, ] %>%
    tidyr::unite("label.txt", all_of(Vendor), sep = "\n") %>%
    .[, .(PULocationID, green, lyft, uber, via, yellow, total, label.txt)]
  
  agg.taxi.all.map <-merge(filtered_demand[, .(PULocationID, total, label.txt)], filtered_supply[, .(DOLocationID, total, label.txt)], 
                           by.x= 'PULocationID', by.y = 'DOLocationID') %>%
    .[, `:=`(total = total.x + total.y,
             total.x = NULL, 
             total.y = NULL)] %>%
    tidyr::unite("label.txt", all_of(c("label.txt.x", "label.txt.y")), sep = "\n\n")
  
  return(list("filtered_supply"=filtered_supply, "filtered_demand"=filtered_demand, "agg.taxi.all.map"=agg.taxi.all.map))
}

##############################
# predefine function for current & vicinity shapes
##############################
prepare_highlight_shape <- function(filtered_demand, filtered_supply, selected.LocationID, agg.taxi.all.map){
  # 1. filter to get the current location as a shape, for later geom_sf
  # 2. look for the vicinities location IDs, they are sorted by distances
  # 3. rbind two vicinity tables of demands and supplies (remember to set factor level), for later barchart
  # 4. look for the least demand and least supply area, color them on x-axis, make suggestion text
  # 5. generate suggested area as shape.
  
  current_area_shape <- filter(NYshape, LocationID == selected.LocationID) %>%
    merge(agg.taxi.all.map[, .(PULocationID, total, label.txt)],
          by.x = "LocationID", by.y = "PULocationID")
  
  distance_at_input <- sort.int(distance_matrix[selected.LocationID, ], index.return = T)
  # remove the inputted location self-distance, which is 0, same as the distance from neighbours
  self_index <- which(distance_at_input$ix %in% c(selected.LocationID, no_tradition_taxi_locations))
  distance_at_input$ix <- distance_at_input$ix[-self_index] # value
  distance_at_input$x <- distance_at_input$x[-self_index] %>% as.numeric() # convert units[m] object to numeric
  vicinity_indices <- distance_at_input$ix[which(distance_at_input$x==0)] %>%
                          c(distance_at_input$ix[1:5])%>%
                          unique()%>%
                          sort()
  
  ##############################
  # select which to go or stay
  ##############################    
  area_comparison_indices <- c(vicinity_indices, selected.LocationID) # vicinity + selected area to show in the plot for comparison
  area_comparison_names <- NYshape$zone[area_comparison_indices]
  
  area_comparison_demand <- filtered_demand[area_comparison_indices, 
                                            !c("label.txt")][, `:=`(
                                              PULocationID = factor(area_comparison_names, levels = area_comparison_names),  # for plotting order
                                              feature="demand")] %>% 
    setnames("PULocationID", "LocationID")
  area_comparison_supply <- filtered_supply[area_comparison_indices, 
                                            !c("label.txt")][, `:=`(
                                              DOLocationID = factor(area_comparison_names, levels = area_comparison_names),
                                              feature="supply")] %>% 
    setnames("DOLocationID", "LocationID")
  area_comparison_table <- list(area_comparison_demand, area_comparison_supply) %>% rbindlist()
  
  demand_ratio <- sort.int((area_comparison_demand$total-area_comparison_supply$total), 
                           decreasing = T,
                           index.return = T)
  best_location <- area_comparison_indices[demand_ratio$ix[1]]

  
  axis_fontColor <- c(rep("black", length(vicinity_indices)), "red")
  axis_fontColor[demand_ratio$ix[1]] <- "blue"
  
  if(best_location==selected.LocationID){
    demand_ratio_suggestion <- "Stay"
  }else{
    demand_ratio_suggestion <- paste("<b style='font-weight:bold;color:blue'>", NYshape$zone[best_location], 
                               "</b> has the largest demand ratio with ",
                               round(demand_ratio$x[1], 2), " demands more than supplies in total.")
  }
  
  vicinity_area_shape <- filter(NYshape, LocationID == best_location) %>%
    merge(agg.taxi.all.map[, .(PULocationID, total, label.txt)],
          by.x = "LocationID", by.y = "PULocationID")
  
  return(list("vicinity_area_shape"=vicinity_area_shape, 
              "current_area_shape"=current_area_shape,
              "demand_ratio_suggestion"=demand_ratio_suggestion, 
              "axis_fontColor"=axis_fontColor,
              "area_comparison_table"=area_comparison_table))
  
}

##############################
# function to plot a barchart for comparison
##############################
comparison_barchart <- function(area_comparison_table, xaxis_fontColor){
  melt(area_comparison_table[, !c("total")], id.vars=c("LocationID", "feature"), variable.name = "Vendor", value.name = "Count")%>%
    ggplot()+
    geom_bar(aes(x=LocationID, y=Count, fill=Vendor), position="stack", stat="identity")+
    labs(title='Predicted demands & supplies at each location')+
    scale_fill_brewer(palette = "Set2")+
    facet_wrap(~feature, ncol = 2)+
    theme(legend.position = "top", 
          legend.title = element_blank(),
          axis.title.x = element_blank(),
          axis.title.y = element_blank(),
          panel.grid.minor = element_blank(), 
          panel.grid.major.x = element_blank(),
          panel.grid.major.y = element_line(colour="grey", size=0.4 , linetype = "dashed"),
          plot.title = element_text(size=10, face="bold", hjust=.5),
          axis.text.x = element_text(angle = 25, vjust = 0.5, hjust=1, family="sans", color = xaxis_fontColor))
}






####################################
# Server                           #
####################################
server <- function(input, output, session) {
  v <- reactiveValues(picked.hm = NULL)
  
  # observeEvent(input$browser,{
  #   browser()
  # })
  # 
  # 
  
  observeEvent(input$Hist_submitbutton, {
    v$pred_yes <- FALSE
    
    v$picked.hm <- as.character(input$Timestamp)%>%strsplit(" ")%>% {.[[1]][2]}
    v$wDay <- input$wDay
    
    mapInputset <- prepare_map_data(v$wDay, v$picked.hm, input$Vendor)
    v$agg.taxi.all.map <- mapInputset$agg.taxi.all.map
    
    # clean the prediction suggestion
    v$axis_fontColor_future <- NULL
    v$axis_fontColor <- NULL
    
  }) # end action Hist_submitbutton 
  
  
  
  observeEvent(input$Pred_submitbutton, {
    v$pred_yes <- TRUE
    
    v$picked.hm <- as.character(input$Timestamp)%>%strsplit(" ")%>% {.[[1]][2]}
    selected.LocationID <- input$pickupID %>% as.numeric()
    v$wDay <- input$wDay
    
    mapInputset <- prepare_map_data(v$wDay, v$picked.hm, input$Vendor)
    suggestion_highlight_set <- prepare_highlight_shape(mapInputset$filtered_demand,
                                                        mapInputset$filtered_supply,
                                                        selected.LocationID, 
                                                        mapInputset$agg.taxi.all.map)
    # for the map
    v$agg.taxi.all.map <- mapInputset$agg.taxi.all.map
    v$vicinity_area_shape <- suggestion_highlight_set$vicinity_area_shape
    v$current_area_shape <- suggestion_highlight_set$current_area_shape
    # for text suggestion at current time frame
    v$demand_ratio_suggestion <- suggestion_highlight_set$demand_ratio_suggestion
    # for barchart comparison at current time frame
    v$area_comparison_table <- suggestion_highlight_set$area_comparison_table
    v$axis_fontColor <- suggestion_highlight_set$axis_fontColor
    
    
    #################
    ## future 10 min barchart
    ################
    future.hm <- as.character(input$Timestamp + 10*60)%>%strsplit(" ")%>% {.[[1]][2]}
    mapInputset <- prepare_map_data(v$wDay, future.hm, input$Vendor)
    suggestion_highlight_set <- prepare_highlight_shape(mapInputset$filtered_demand,
                                                        mapInputset$filtered_supply,
                                                        selected.LocationID, 
                                                        mapInputset$agg.taxi.all.map)
    # for text suggestion at future time frame
    v$demand_ratio_suggestion_future <- suggestion_highlight_set$demand_ratio_suggestion
    # for barchart comparison at future time frame
    v$area_comparison_table_future <- suggestion_highlight_set$area_comparison_table
    v$axis_fontColor_future <- suggestion_highlight_set$axis_fontColor
    
  }) # end action Pred_submitbutton
  
  
  
  ############################################################
  #####################  Make the map    #####################
  ############################################################
  output$distMapPlot <- renderPlotly({
    
    if (is.null(v$agg.taxi.all.map)) return()
    
    p <- merge(NYshape, v$agg.taxi.all.map[, .(PULocationID, total, label.txt)],
               by.x = "LocationID", by.y = "PULocationID") %>%
      ggplot() +
      geom_sf(aes(fill = total, text = label.txt), lwd = 0) +
      scale_fill_continuous(low = "#c9e9e0",
                            high = "#366791",
                            na.value = "white") +
      theme_bw() +
      theme_void() +
      theme(legend.position = c(1.05, .25))
    
    if(v$pred_yes){
      p <- p + geom_sf(data = v$current_area_shape,
                       aes(fill = total, text = label.txt), lwd = 0.3, color = "#fb3640")+ # outline current location
        geom_sf(data = v$vicinity_area_shape,
                aes(text = label.txt), fill = "#fdca40", lwd = 0)+ # fill yellow for suggesting area to go
        labs(
          title = paste0("NY predicted demands and supplies at <span style='color:#009E73;'>",
                         v$picked.hm,
                         "</span> on every <span style='color:#009E73;'>",
                         v$wDay,"</span>"))
    }else{
      p <- p + labs(
        title = paste0("NY historical orders at <span style='color:#009E73;'>",
                       v$picked.hm,
                       "</span> on every <span style='color:#009E73;'>",
                       v$wDay,"</span>"))
    }
    
    ggplotly(p)
    
  }) # end renderPlotly for interactive
  
  
  
  output$suggestions_curr <- renderUI({
    if (is.null(v$axis_fontColor)) return()
    str <- "<b style='font-weight:bold;color:#fb3640'>Red boundary area</b> is your current place.<br/>"
    str <- c(str, "I suggest you to go to the <b style='font-weight:bold;color:#fdca40'>Yellow filled area</b> based on the following details:<br/>")
    str <- c(str, '<br/>Suggestion based on total amount of cabs:<br/>', paste("<b style='font-weight:bold;'>Best</b> suggestion:", v$demand_ratio_suggestion, '<br/>'))
    HTML(str)
  })
  
  output$suggestions_future <- renderUI({
    if (is.null(v$axis_fontColor_future)) return()
    str <- paste("<b style='font-weight:bold;'>Best</b> suggestion for <b style='font-weight:bold;'>future 10min</b>:",
                 v$demand_ratio_suggestion_future, '<br/>')
    HTML(str)
  })
  
  ############################################################
  ##################   Make the barchart    ##################
  ############################################################
  output$distStatPlot_curr <- renderPlot({
    if (is.null(v$axis_fontColor)) return()
    comparison_barchart(v$area_comparison_table, v$axis_fontColor)
  }) # end of renderPlot for barchart
  
  output$distStatPlot_future <- renderPlot({
    if (is.null(v$axis_fontColor_future)) return()
    comparison_barchart(v$area_comparison_table_future, v$axis_fontColor_future)
  }) # end of renderPlot for barchart
  
  
  
  output$aboutHTML <- renderUI(includeMarkdown("./about.md"))
  
}


####################################
# Create Shiny App                 #
####################################
shinyApp(ui = ui, server = server, options = list(port = 7859))



Writing shiny_app_colab.r


In [2]:
%%writefile about.md
# Main features

This web app displays the historical records of Uber/Lyft/Via and Green & Yellow taxies in New York city in 2019. 
And based on these records, we built an LSTM neural network model to predict next week's cab demands and supplies.
Since the records are too large to load, we preprocess them into aggregated data tables, and shows only on requested features such as 
"weekdays", "time period of the day", "taxi vendor", etc. What you then see is a map plotted on the filtered data.

In light of our model's prediction power, since it is a time series model, we couldn't use accuracy for evaluation.
Below are example comparisons for uber between true target values and predicted values on some locations. 

![uber demand true vs predict](https://t1.picb.cc/uploads/2021/04/30/ZD8lcD.png)

![uber supply true vs predict](https://t1.picb.cc/uploads/2021/04/30/ZD8uQv.png)



Writing about.md


## Prerequisite - Setup Ubuntu & R evironment for spatial data

It takes 5min 10~20s

In [3]:
%load_ext rpy2.ipython

In [4]:
%%capture
# add ubuntuGIS repository
!sudo add-apt-repository -y ppa:ubuntugis/ubuntugis-unstable
!sudo apt-get update
# R spatial environment
!sudo apt install -y libudunits2-dev libgdal-dev libgeos-dev libproj-dev
!sudo apt install -y r-cran-shiny r-cran-shinythemes r-cran-shinycssloaders r-cran-plotly

# ngrok intranet penetration
!pip -q install pyngrok

# RData which contains predefined variables
!gdown --id '1m1_Ub9adla8v751GQi1vBjlYtkDZN5fq'

In [5]:
%%R
install.packages("rgdal") # required for sf, dependency of libgdal-dev
install.packages("sf")

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependency ‘sp’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/sp_1.4-5.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 1049195 bytes (1.0 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]:

## Run the app online with ngrok

If the map does not show up, <font color='yellow'>refresh the web page</font>. 

In [7]:
from pyngrok import ngrok

public_url = ngrok.connect(addr='7859')
print(public_url)

# from IPython.display import Javascript
# url = "window.open('{}')".format(public_url.public_url)
# Javascript(url)

!R -e "shiny::runApp('/content/shiny_app_colab.r')"

NgrokTunnel: "http://395f3085748e.ngrok.io" -> "http://localhost:7859"

R version 4.1.0 (2021-05-18) -- "Camp Pontanezen"
Copyright (C) 2021 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> shiny::runApp('/content/shiny_app_colab.r')
Loading required package: shiny

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

   